In [4]:
# Import dependencies
import pandas as pd
import chardet
# For data base creation
from sqlalchemy import create_engine
# from config import db_password
import os
import os.path, sys
import glob
import re

In [3]:
# tournament file location
file = '../../atp-world-tour-tennis-data/csv/1_tournaments/'

In [4]:
# def for cleaning tournament data
def clean_tournament(file):
    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        with open(file+files[i], 'rb') as rawdata:
            result = chardet.detect(rawdata.read(100000))
        temp_df = pd.read_csv(file+files[i], encoding=result['encoding'], header=None)
        dataframes_list.append(temp_df)

    df = pd.concat(dataframes_list)
    df.drop(columns=[1,4,5,8,9,10,12,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30], inplace=True)
    df = df.reindex(columns=[0,2,3,7,6,11,13,14,22])
    header_df = pd.read_csv("../temp/Tourney_header.csv")
    df.columns = header_df.columns
    #Combinding coloumns 2,3
    df["Name"] = df["Name"]+"-"+ df["temp"]
    df = df.drop(columns =['temp'])
    df = df.reset_index()
    return df

In [5]:
# running CT function
df = clean_tournament(file)
df.head(20)

,index,Turney_id,Name,Date,Location,Draw_size,Conditions,Surface,Winner_id
0,0,1877-540,Grand Slam-Wimbledon,1877.07.09,"London, Great Britain",32,Outdoor,Grass,gi91
1,1,1878-540,Grand Slam-Wimbledon,1878.07.08,"London, Great Britain",64,Outdoor,Grass,hg50
2,2,1879-540,Grand Slam-Wimbledon,1879.07.07,"London, Great Britain",64,Outdoor,Grass,hg35
3,3,1880-540,Grand Slam-Wimbledon,1880.07.05,"London, Great Britain",64,Outdoor,Grass,hg35
4,4,1881-540,Grand Slam-Wimbledon,1881.07.02,"London, Great Britain",64,Outdoor,Grass,rg71
5,5,1881-560,Grand Slam-US Championships,1881.08.31,"Newport, United States",32,Outdoor,Grass,gj02
6,6,1882-540,Grand Slam-Wimbledon,1882.07.08,"London, Great Britain",32,Outdoor,Grass,rg71
7,7,1882-560,Grand Slam-US Championships,1882.08.30,"Newport, United States",64,Outdoor,Grass,cm63
8,8,1883-540,Grand Slam-Wimbledon,1883.07.07,"London, Great Britain",32,Outdoor,Grass,rg71
9,9,1883-560,Grand Slam-US Championships,1883.08.21,"Newport, United States",32,Outdoor,Grass,dg24


In [10]:
def clean_Match_Score(file):
    # finds information and puts it in a df
    file = '../../atp-world-tour-tennis-data/csv/2_match_scores/'

    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        temp_df = pd.read_csv(file+files[i], encoding='utf-8', header=None)
        dataframes_list.append(temp_df)

    df_W = pd.concat(dataframes_list)
    df_L = pd.concat(dataframes_list)
    # dropping un needed winner stats
    df_W.drop(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,21,22,24,25,26,28,29,30,31,32,33], inplace=True)
    # Dropping un needed Loser stats
    df_L.drop(columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,21,22,24,25,26,28,29,30,31,32,33,35], inplace=True)
    # reorder columns to match headers
    df_W =df_W.reindex(columns = [34,0,20,16,27,5,23])
    df_L=df_L.reindex(columns=[34,0,23,16,27,5,20])
    # setting winner&Loser headers
    df_W_Header = pd.read_csv("../temp/Match_score_header_W.csv")
    df_L_Header = pd.read_csv("../temp/Match_score_header_L.csv")
    # connecting winner&loser headers
    df_W[34] = pd.Series(df_W[0].astype(str) + '-' + df_W[20].astype(str) + '-' + df_W[23].astype(str), index=df_W.index)
    df_L[34] = pd.Series(df_W[0].astype(str) + '-' + df_W[20].astype(str) + '-' + df_W[23].astype(str), index=df_W.index)
    df_W.drop(columns=[23], inplace=True)
    df_L.drop(columns= [20], inplace=True)
    df_W.columns = df_W_Header.columns
    df_L.columns = df_L_Header.columns
    # Setting player_results
    df_W['Player_result'] = pd.Series('Won', index=df_W.index)
    df_L['Player_result'] = pd.Series('Lost', index=df_L.index)
    # Combining winner and loser dataframes
    df = pd.concat([df_W, df_L])
    df = df.reset_index()
    replace_values = {'61': '6-1', '62': '6-2', '63': '6-3', '64': '6-4', '65': '6-5', '16': '1-6', '26': '2-6', '36': '3-6', '46': '4-6', '56': '5-6', '76':'7-6', '67': '6-7', \
                      '57': '5-7', '75': '7-5', '68': '6-8', '86': '8-6', '79': '7-9', '97': '9-7', '108': '10-8', '119': '11-9', \
                      '1210': '12-10', '1311': '13-11', '1412': '14-12', '1513': '15-13', '1614': '16-14', \
                      '1715': '17-15', '1816': '18-16', '1917': '19-17','2018': '20-18', \
                      '2119': '21-19', '2220': '22-20', '2321': '23-21', '2422': '24-22', '2523': '25-23', \
                      '2624': '26-24', '7068': '70-68', '06': '0-6', '60': '6-0'}
    for key, value in replace_values.items():
        df['Score'] = df['Score'].str.replace(key, value)
    return df

In [12]:
df = clean_Match_Score(file)
df.tail(20)

,index,Match_id,Tourney_id,Player_id,Round,Score,Player_result
446154,9865,2023-8998-d0co-su87,2023-8998,su87,Round of 32,6-4 6-2,Lost
446155,9866,2023-8998-h756-bm95,2023-8998,bm95,Round of 32,3-6 7-6(8) 7-6(3),Lost
446156,9867,2023-8998-hh06-rc91,2023-8998,rc91,Round of 32,4-6 7-6(5) 6-4,Lost
446157,9868,2023-8998-kd46-p09z,2023-8998,p09z,Round of 32,6-0 6-7(5) 7-5,Lost
446158,9869,2023-8998-kb95-ea24,2023-8998,ea24,Round of 32,7-6(2) 7-5,Lost
446159,9870,2023-8998-kf17-m0fh,2023-8998,m0fh,Round of 32,6-4 6-4,Lost
446160,9871,2023-8998-mh30-r772,2023-8998,r772,Round of 32,4-6 6-3 7-6(6),Lost
446161,9872,2023-8998-y268-rh24,2023-8998,rh24,Round of 32,6-4 4-6 6-3,Lost
446162,9873,2023-8998-y268-tc61,2023-8998,tc61,2nd Round Qualifying,7-6(4) 7-5,Lost
446163,9874,2023-8998-mh30-o483,2023-8998,o483,2nd Round Qualifying,6-3 7-6(7),Lost


In [1]:
def clean_match_stats(file):
    pd.options.display.max_columns = 59
    # finds informations and puts it in a df
    file = '../../atp-world-tour-tennis-data/csv/3_match_stats/'

    # assign path
    path, dirs, files = next(os.walk(file))
    file_count = len(files)
    # create empty list
    dataframes_list = []
    
    # append datasets to the list
    for i in range(file_count):
        temp_df = pd.read_csv(file+files[i], encoding='utf-8', header=None)
        dataframes_list.append(temp_df)

    df_W = pd.concat(dataframes_list)
    df_L = pd.concat(dataframes_list)
    df_W.drop(columns=[1,4,6,11,12,18,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58], inplace=True)
    df_L.drop(columns=[1,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,38,39,45], inplace=True)
    df_W[0] = pd.Series(df_W[0].astype(str).str.split('-',2).str[:2].str.join('-'))
    df_W[0] = pd.Series(df_W[0].astype(str) + '-' + df_W[5].astype(str) + '-' + df_W[32].astype(str))
    df_L[0] = pd.Series(df_L[0].astype(str).str.split('-',2).str[:2].str.join('-'))
    df_L[0] = pd.Series(df_L[0].astype(str) + '-' + df_L[5].astype(str) + '-' + df_L[32].astype(str))
    year_pattern = re.compile(r'(?:20(20|21|22|23))')
    df_L[0] = df_L[0].apply(lambda x: x.split('-')[0] +'-' if year_pattern.search(str(x)) else x)
    df_L[2] = df_L[2].apply(lambda x: x.split('/')[6] if year_pattern.search(str(x)) else '')
    df_L.loc[df_L[0].str.contains('2022-'), 0] = df_L.loc[df_L[0].str.contains('2022-'), 0].astype(str) + df_L.loc[df_L[0].str.contains('2022-'), 2].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2022-'), 5].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2022-'), 32].astype(str)
    df_L.loc[df_L[0].str.contains('2021-'), 0] = df_L.loc[df_L[0].str.contains('2021-'), 0].astype(str) + df_L.loc[df_L[0].str.contains('2021-'), 2].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2021-'), 5].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2021-'), 32].astype(str)
    df_L.loc[df_L[0].str.contains('2020-'), 0] = df_L.loc[df_L[0].str.contains('2020-'), 0].astype(str) + df_L.loc[df_L[0].str.contains('2020-'), 2].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2020-'), 5].astype(str) + '-' + df_L.loc[df_L[0].str.contains('2020-'), 32].astype(str)
    df_W[0] = df_W[0].apply(lambda x: x.split('-')[0] +'-' if year_pattern.search(str(x)) else x)
    df_W[2] = df_W[2].apply(lambda x: x.split('/')[6] if year_pattern.search(str(x)) else '')
    df_W.loc[df_W[0].str.contains('2022-'), 0] = df_W.loc[df_W[0].str.contains('2022-'), 0].astype(str) + df_W.loc[df_W[0].str.contains('2022-'), 2].astype(str) + '-' + df_W.loc[df_L[0].str.contains('2022-'), 5].astype(str) + '-' + df_W.loc[df_W[0].str.contains('2022-'), 32].astype(str)
    df_W.loc[df_W[0].str.contains('2021-'), 0] = df_W.loc[df_W[0].str.contains('2021-'), 0].astype(str) + df_W.loc[df_W[0].str.contains('2021-'), 2].astype(str) + '-' + df_W.loc[df_L[0].str.contains('2021-'), 5].astype(str) + '-' + df_W.loc[df_W[0].str.contains('2021-'), 32].astype(str)
    df_W.loc[df_W[0].str.contains('2020-'), 0] = df_W.loc[df_W[0].str.contains('2020-'), 0].astype(str) + df_W.loc[df_W[0].str.contains('2020-'), 2].astype(str) + '-' + df_W.loc[df_L[0].str.contains('2020-'), 5].astype(str) + '-' + df_W.loc[df_W[0].str.contains('2020-'), 32].astype(str)
    df_W.drop(columns=[32,2], inplace=True)
    df_L.drop(columns=[5,2], inplace=True)
    df_W =df_W.reindex(columns = [0,5,3,7,8,9,10,13,14,15,16,26,27,19,20,21,22,23,24,25,17,28,29,30,31])
    df_L =df_L.reindex(columns=[0,32,3,34,35,36,37,40,41,42,43,53,54,46,47,48,49,50,51,52,44,55,56,57,58])
    df_W_Header = pd.read_csv('../temp/Match_stats_header.csv')
    df_W.columns = df_W_Header.columns
    df_L.columns = df_W_Header.columns
    df = pd.concat([df_W, df_L])
    df.dropna(axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [5]:
df = clean_match_stats(file)
df

C:\Users\jspim\AppData\Local\Temp\ipykernel_6212\2307069082.py:21: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_W[0] = pd.Series(df_W[0].astype(str).str.split('-',2).str[:2].str.join('-'))
C:\Users\jspim\AppData\Local\Temp\ipykernel_6212\2307069082.py:23: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_L[0] = pd.Series(df_L[0].astype(str).str.split('-',2).str[:2].str.join('-'))


,Match_id,Player_id,Duration,Aces,Double_faults,First_serves_in,First_serves_total,Second_serve_points_won,Second_serve_points_total,Break_points_saved,Break_points_serve_total,Service_points_won,Service_points_total,First_serves_return_won,First_serves_return_total,Second_serves_return_won,Second_serves_return_total,Break_points_converted,Break_point_return_total,Service_games_played,Returns_games_played,Return_points_won,Return_points_total,Total_points_won,Total_points_total
0,1991-7308-k181-s351,k181,01:29:00,1.0,2.0,44.0,65.0,12.0,21.0,2.0,5.0,43.0,65.0,16.0,47.0,21.0,36.0,4.0,7.0,12.0,12.0,37.0,83.0,80.0,148.0
1,1991-7308-s351-c243,s351,01:44:00,7.0,4.0,37.0,80.0,27.0,43.0,5.0,6.0,53.0,80.0,11.0,40.0,13.0,31.0,2.0,3.0,11.0,11.0,24.0,71.0,77.0,151.0
2,1991-7308-k181-l206,k181,01:20:00,3.0,0.0,0.0,65.0,12.0,19.0,0.0,2.0,41.0,65.0,12.0,36.0,20.0,33.0,4.0,7.0,11.0,11.0,32.0,69.0,73.0,134.0
3,1991-7308-c243-s367,c243,01:20:00,5.0,2.0,41.0,61.0,14.0,20.0,0.0,1.0,45.0,61.0,6.0,34.0,15.0,27.0,2.0,3.0,10.0,11.0,21.0,61.0,66.0,122.0
4,1991-7308-s351-a031,s351,01:30:00,5.0,2.0,45.0,74.0,18.0,29.0,2.0,4.0,52.0,74.0,18.0,57.0,12.0,24.0,3.0,7.0,13.0,14.0,30.0,81.0,82.0,155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208125,2022-560-ke29-ke17,ke17,03:39:00,31.0,5.0,115.0,160.0,21.0,45.0,4.0,8.0,112.0,160.0,23.0,99.0,24.0,67.0,2.0,9.0,28.0,28.0,47.0,166.0,159.0,326.0
208126,2022-560-ke17-mm58,mm58,02:53:00,22.0,2.0,75.0,127.0,23.0,52.0,2.0,7.0,79.0,127.0,21.0,85.0,19.0,35.0,3.0,8.0,19.0,19.0,40.0,120.0,119.0,247.0
208127,2022-560-td51-n409,n409,03:34:00,9.0,9.0,60.0,114.0,30.0,54.0,3.0,8.0,73.0,114.0,14.0,58.0,23.0,58.0,2.0,6.0,19.0,20.0,37.0,116.0,110.0,230.0
208128,2022-560-a0e2-c977,c977,03:54:00,14.0,13.0,73.0,142.0,33.0,69.0,12.0,18.0,87.0,142.0,39.0,103.0,16.0,40.0,5.0,9.0,24.0,24.0,55.0,143.0,142.0,285.0
